In [1]:
import pandas as pd
import numpy as np
import atomsci.ddm.pipeline.model_pipeline as mp
import atomsci.ddm.pipeline.parameter_parser as parse
import atomsci.ddm.utils.curate_data as curate_data
import atomsci.ddm.utils.struct_utils as struct_utils
from atomsci.ddm.pipeline import perf_plots as pp
from atomsci.ddm.pipeline import perf_data
from atomsci.ddm.pipeline import compare_models as cm


import os, json, sys, glob, pickle
import matplotlib.pyplot as plt
import seaborn as sns
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

2022-01-27 12:24:48.682273: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /apps/spack/brown/apps/proj/5.2.0-gcc-4.8.5-usib7od/lib:/apps/spack/brown/apps/geos/3.7.2-gcc-4.8.5-3vanyva/lib:/apps/spack/brown/apps/gdal/2.4.2-gcc-4.8.5-uj736h3/lib:/apps/spack/brown/apps/netcdf/4.5.0-gcc-6.3.0-d6fczmr/lib:/apps/spack/brown/apps/libtiff/4.0.10-gcc-6.3.0-6p5trqs/lib:/apps/spack/brown/apps/hdf5/1.8.16-gcc-6.3.0-7q7ndrz/lib:/apps/spack/brown/apps/hdf/4.2.14-gcc-6.3.0-2xg7pyg/lib:/scratch/brown/kamstut/tdm/apps/jupyter/kernels/llnl/.venv/lib:/apps/spack/brown/apps/r/4.0.0-gcc-6.3.0-hrvmcqp/rlib/R/lib:/apps/spack/brown/apps/openblas/0.3.7-gcc-6.3.0-qk24sho/lib:/apps/spack/brown/apps/tk/8.6.8-gcc-6.3.0-6qaesqb/lib:/apps/spack/brown/apps/tcl/8.6.8-gcc-6.3.0-n6mxabo/lib:/apps/spack/brown/apps/zlib/1.2.11-gcc-4.8.5-pkmj6e7/lib:/app

In [2]:
split_uuid = '/home/pmoulik/OPRD1sp22/OPRD1_sp22_train_valid_test_scaffold_ebfd92e1-3ad0-43e4-9139-7e215ca55a45'
response_col = "target_OPRD1_standard_value"
train_file = '/home/pmoulik/OPRD1_sp22.csv'
df=pd.read_csv(train_file)
df.rename(columns={'compound_id' : 'SMILES'},inplace=True)
df['compound_id']=df.index
ofile='/home/pmoulik/OPRD1_sp22.csv'
df.to_csv(ofile,index=False)
train_file=ofile
compound_id = "compound_id"
smiles_col = "SMILES"

params = {
        "system": "LC",
        "lc_account": 'None',
        "datastore": "False",
        "save_results": "False",
        "prediction_type": "regression",
        "dataset_key": train_file,
        "id_col": compound_id,
        "smiles_col": smiles_col,
        "response_cols": response_col,
        "split_uuid": split_uuid,
        "previously_split": "False",
        "featurizer": "ecfp",
        "descriptor_type": "rdkit_raw",
        "model_type": "RF", # different model type 
        "verbose": "True",
        "transformers": "True",
        'max_epochs': '70',
        "rerun": "False",
        "result_dir": "/home/pmoulik/RF_sp22/rdkit-raw/"
    }

In [3]:
rd_param = parse.wrapper(params)
rd = mp.ModelPipeline(rd_param)
rd.train_model()

2022-01-27 12:27:05,811 Created a dataset hash 'e0ad2cf61651f72b9c351ac059c34b72' from dataset_key '/home/pmoulik/OPRD1_sp22.csv'
2022-01-27 12:27:05,820 previously_split is set False; ignoring split_uuid passed as parameter
2022-01-27 12:27:05,822 Attempting to load featurized dataset
2022-01-27 12:27:05,865 Exception when trying to load featurized data:
DynamicFeaturization doesn't support get_featurized_dset_name()
2022-01-27 12:27:05,870 Featurized dataset not previously saved for dataset OPRD1_sp22, creating new
2022-01-27 12:27:06,200 Featurizing sample 0
2022-01-27 12:27:09,423 Featurizing sample 1000
2022-01-27 12:27:12,493 Featurizing sample 2000
2022-01-27 12:27:15,567 Featurizing sample 3000
2022-01-27 12:27:16,232 Splitting data by scaffold


number of features: 1024


2022-01-27 12:27:18,627 Dataset split table saved to /home/pmoulik/OPRD1_sp22_train_valid_test_scaffold_c9ca62a1-6fb6-4621-8223-e117eda9e842.csv
2022-01-27 12:27:18,674 Wrote transformers to /home/pmoulik/RF_sp22/rdkit-raw/OPRD1_sp22/RF_ecfp_scaffold_regression/858ec84e-a24a-48e8-b3e8-ea31f1c0141f/transformers.pkl
2022-01-27 12:27:18,676 Transforming response data
2022-01-27 12:27:18,716 Transforming response data
2022-01-27 12:27:18,727 Transforming response data
2022-01-27 12:27:18,737 Fitting random forest model
2022-01-27 12:27:26,465 Fold 0: training r2_score = 0.965, validation r2_score = 0.747, test r2_score = 0.694
2022-01-27 12:27:28,809 Wrote model tarball to /home/pmoulik/RF_sp22/rdkit-raw/OPRD1_sp22_model_858ec84e-a24a-48e8-b3e8-ea31f1c0141f.tar.gz


In [4]:
rfe_choices = [16, 32, 64, 128] # setting the options for our parameters 
rfd_choices = [16, 32, 64, 128]
rff_choices = [16, 32, 64, 128]


rfe_list = [] # estimators
rfd_list = [] # depth
rff_list = [] # features 
valid_r2_list = [] # creating a list for the valid r2 scores 
for rf_estimator in rfe_choices:
    for rf_depth in rfd_choices:
        for rf_feature in rff_choices:
            params["rf_estimators"] = rf_estimator # choosing an estimator from our parameter list (param_choice) 
            params["rf_max_depth"] = rf_depth # choosing depth for the max depth
            params["rf_max_features"] = rf_feature # choosing a number for the max features 
            rfe_list.append(rf_estimator) # appending the chosen estimator to our estimator list
            rfd_list.append(rf_depth) # appending the chosen depth to our depth list
            rff_list.append(rf_feature) # appending the chosen feature to our feature list 
            tp = parse.wrapper(params) # wrapping using ATOM function
            pl = mp.ModelPipeline(tp) # running through the model pipeline [from ATOM]
            pl.train_model() # training our models based on these parameters 
            pred_data = pl.model_wrapper.get_perf_data(subset="valid", epoch_label="best") # grabbing best model
            pred_results = pred_data.get_prediction_results() # predicting results from the best model
            valid_r2 = pred_results['r2_score'] # extracting the r2 validation score from the best model
            valid_r2_list.append(valid_r2) # appending the r2 score to our list 
            print(f"rf_estimators: {rf_estimator}, rf_max_depth: {rf_depth}, rf_max_features: {rf_feature}, valid_r2: {valid_r2}") # printing out the used estiamtors, depth, and features and the resulting r2 score

2022-01-27 12:28:18,775 Created a dataset hash 'e0ad2cf61651f72b9c351ac059c34b72' from dataset_key '/home/pmoulik/OPRD1_sp22.csv'
2022-01-27 12:28:18,785 previously_split is set False; ignoring split_uuid passed as parameter
2022-01-27 12:28:18,788 Attempting to load featurized dataset
2022-01-27 12:28:18,801 Exception when trying to load featurized data:
DynamicFeaturization doesn't support get_featurized_dset_name()
2022-01-27 12:28:18,804 Featurized dataset not previously saved for dataset OPRD1_sp22, creating new
2022-01-27 12:28:18,817 Featurizing sample 0
2022-01-27 12:28:21,967 Featurizing sample 1000
2022-01-27 12:28:25,040 Featurizing sample 2000
2022-01-27 12:28:28,083 Featurizing sample 3000
2022-01-27 12:28:28,736 Splitting data by scaffold


number of features: 1024


2022-01-27 12:28:30,772 Dataset split table saved to /home/pmoulik/OPRD1_sp22_train_valid_test_scaffold_205d324e-6d5d-44f1-8a12-b85c2c542f37.csv
2022-01-27 12:28:30,816 Wrote transformers to /home/pmoulik/RF_sp22/rdkit-raw/OPRD1_sp22/RF_ecfp_scaffold_regression/343ad0e5-c5bc-49a8-aea7-b601e9f729ec/transformers.pkl
2022-01-27 12:28:30,818 Transforming response data
2022-01-27 12:28:30,849 Transforming response data
2022-01-27 12:28:30,860 Transforming response data
2022-01-27 12:28:30,871 Fitting random forest model
2022-01-27 12:28:31,059 Fold 0: training r2_score = 0.871, validation r2_score = 0.683, test r2_score = 0.617
2022-01-27 12:28:31,112 Wrote model tarball to /home/pmoulik/RF_sp22/rdkit-raw/OPRD1_sp22_model_343ad0e5-c5bc-49a8-aea7-b601e9f729ec.tar.gz
2022-01-27 12:28:31,154 Created a dataset hash 'e0ad2cf61651f72b9c351ac059c34b72' from dataset_key '/home/pmoulik/OPRD1_sp22.csv'
2022-01-27 12:28:31,161 previously_split is set False; ignoring split_uuid passed as parameter
2022

rf_estimators: 16, rf_max_depth: 16, rf_max_features: 16, valid_r2: 0.6826156755448965


2022-01-27 12:28:34,209 Featurizing sample 1000
2022-01-27 12:28:37,267 Featurizing sample 2000
2022-01-27 12:28:40,433 Featurizing sample 3000
2022-01-27 12:28:41,084 Splitting data by scaffold


...

rf_estimators: 128, rf_max_depth: 128, rf_max_features: 64, valid_r2: 0.755096656993143


2022-01-27 12:42:29,340 Featurizing sample 1000
2022-01-27 12:42:32,410 Featurizing sample 2000
2022-01-27 12:42:35,469 Featurizing sample 3000
2022-01-27 12:42:36,124 Splitting data by scaffold


number of features: 1024


2022-01-27 12:42:38,124 Dataset split table saved to /home/pmoulik/OPRD1_sp22_train_valid_test_scaffold_6edc2c60-b2dc-4bf1-9bdb-6028bc16ad92.csv
2022-01-27 12:42:38,184 Wrote transformers to /home/pmoulik/RF_sp22/rdkit-raw/OPRD1_sp22/RF_ecfp_scaffold_regression/5b9f14b7-8096-46b9-ad02-f94f6c4e2840/transformers.pkl
2022-01-27 12:42:38,186 Transforming response data
2022-01-27 12:42:38,226 Transforming response data
2022-01-27 12:42:38,237 Transforming response data
2022-01-27 12:42:38,248 Fitting random forest model
2022-01-27 12:42:41,844 Fold 0: training r2_score = 0.965, validation r2_score = 0.759, test r2_score = 0.701
2022-01-27 12:42:42,573 Wrote model tarball to /home/pmoulik/RF_sp22/rdkit-raw/OPRD1_sp22_model_5b9f14b7-8096-46b9-ad02-f94f6c4e2840.tar.gz


rf_estimators: 128, rf_max_depth: 128, rf_max_features: 128, valid_r2: 0.758992372925428


In [5]:
odir = '/home/pmoulik/RF_sp22/rdkit-raw/'

In [6]:
results_df = cm.get_filesystem_perf_results(odir, pred_type='regression')

Found data for 130 models under /home/pmoulik/RF_sp22/rdkit-raw/


In [7]:
results_OPRD1 = results_df.loc[results_df['dataset_key'] == "/home/pmoulik/OPRD1_sp22.csv"]

In [8]:
results_OPRD1.sort_values(["valid_r2_score"], ascending=[False])

,model_uuid,model_path,ampl_version,model_type,dataset_key,featurizer,splitter,model_score_type,feature_transform_type,rf_estimators,...,train_mae_score,train_num_compounds,valid_r2_score,valid_rms_score,valid_mae_score,valid_num_compounds,test_r2_score,test_rms_score,test_mae_score,test_num_compounds
109,13f73c61-7891-4075-83b2-d30ff13ada46,/home/pmoulik/RF_sp22/rdkit-raw/OPRD1_sp22_mod...,1.3.0,RF,/home/pmoulik/OPRD1_sp22.csv,ecfp,scaffold,r2,normalization,128,...,0.176619,2559,0.760002,0.717437,0.531295,320,0.702302,0.771359,0.576082,320
111,5b9f14b7-8096-46b9-ad02-f94f6c4e2840,/home/pmoulik/RF_sp22/rdkit-raw/OPRD1_sp22_mod...,1.3.0,RF,/home/pmoulik/OPRD1_sp22.csv,ecfp,scaffold,r2,normalization,128,...,0.176520,2559,0.758992,0.718945,0.531493,320,0.700897,0.773177,0.576700,320
98,9398d625-0a89-4b6e-88e6-f06cfed9a45d,/home/pmoulik/RF_sp22/rdkit-raw/OPRD1_sp22_mod...,1.3.0,RF,/home/pmoulik/OPRD1_sp22.csv,ecfp,scaffold,r2,normalization,128,...,0.266565,2559,0.755991,0.723408,0.545546,320,0.692935,0.783400,0.586484,320
127,870a5a57-e4df-40ec-84e1-b03e110af5fe,/home/pmoulik/RF_sp22/rdkit-raw/OPRD1_sp22_mod...,1.3.0,RF,/home/pmoulik/OPRD1_sp22.csv,ecfp,scaffold,r2,normalization,128,...,0.177622,2559,0.755683,0.723865,0.539714,320,0.699081,0.775521,0.579428,320
101,97a58786-9c3b-42a3-943e-b49268fea910,/home/pmoulik/RF_sp22/rdkit-raw/OPRD1_sp22_mod...,1.3.0,RF,/home/pmoulik/OPRD1_sp22.csv,ecfp,scaffold,r2,normalization,128,...,0.177656,2559,0.755097,0.724732,0.540837,320,0.698494,0.776276,0.579562,320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,829c2357-7cb1-44fa-8862-2ba82718b799,/home/pmoulik/RF_sp22/rdkit-raw/OPRD1_sp22_mod...,1.3.0,RF,/home/pmoulik/OPRD1_sp22.csv,ecfp,scaffold,r2,normalization,16,...,0.292988,2559,0.714841,0.782030,0.590016,320,0.641784,0.846137,0.629644,320
73,d806b552-bbe3-48fd-8fd2-752633a7d361,/home/pmoulik/RF_sp22/rdkit-raw/OPRD1_sp22_mod...,1.3.0,RF,/home/pmoulik/OPRD1_sp22.csv,ecfp,scaffold,r2,normalization,64,...,0.393726,2559,0.689672,0.815814,0.645031,320,0.617067,0.874842,0.698975,320
85,ef7ba80c-b763-4142-8037-f64d07345889,/home/pmoulik/RF_sp22/rdkit-raw/OPRD1_sp22_mod...,1.3.0,RF,/home/pmoulik/OPRD1_sp22.csv,ecfp,scaffold,r2,normalization,128,...,0.389250,2559,0.687024,0.819286,0.643173,320,0.618935,0.872706,0.695611,320
78,7030867d-3856-4247-9220-961a37217b3f,/home/pmoulik/RF_sp22/rdkit-raw/OPRD1_sp22_mod...,1.3.0,RF,/home/pmoulik/OPRD1_sp22.csv,ecfp,scaffold,r2,normalization,32,...,0.390795,2559,0.685633,0.821105,0.651557,320,0.614706,0.877535,0.699058,320
